In [117]:
#%pip install genson

In [118]:


instrument = 'SPX500'
ifn = instrument.replace('/', '-')
timeframe = 'D1'

fn_basename = 'samples/' + ifn + '_'+timeframe+'.targets.mx'




In [119]:


import pandas as pd
from genson import SchemaBuilder



# Prep

In [120]:

# filter the data and select first

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(fn_basename + '.csv')

# Select the desired columns
columns_to_select = ['Date', 'ao', 'jaw', 'teeth', 'lips', 'fdbb', 'price_peak_bellow', 'ao_peak_bellow', 'target']
df_filtered = df[columns_to_select]

# Rename the 'Date' column to 'timestamp'
df_filtered = df_filtered.rename(columns={'Date': 'timestamp'})

# timestamp as type timestamp
df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])

# Rename the 'target' column to 'target_value'
df_filtered = df_filtered.rename(columns={'target': 'target_value'})


import tlid
# Add a new column 'item_id'
df_filtered['item_id'] = instrument + '_' + timeframe + '_' + df_filtered['timestamp'].apply(lambda x: tlid.fromdtstr(str(x)))

# Move item_id to the first column
cols = df_filtered.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_filtered = df_filtered[cols]


# Move target_value to the third column

cols = df_filtered.columns.tolist()
cols.insert(2, cols.pop(cols.index('target_value')))
df_filtered = df_filtered[cols]




In [121]:
# set 'target_value' is type : float
df_filtered['target_value'] = df_filtered['target_value'].astype(float)




In [122]:


# Write the filtered DataFrame to a file
filtered_filename = fn_basename + '.filtered_data.csv'

df_filtered.to_csv(filtered_filename, index=False)

df = df_filtered

In [123]:
# Convert the DataFrame into a JSON object
json_obj = df.to_dict(orient='records')

# Use the genson library to generate a JSON schema
builder = SchemaBuilder()
builder.add_schema({"type": "object", "properties": {}})
for row in json_obj:
    # Convert the Timestamp object to a string
    row['timestamp'] = str(row['timestamp'])
    builder.add_object(row)

# Print the JSON schema
buildder_schema_data = builder.to_schema()
print(buildder_schema_data)


{'$schema': 'http://json-schema.org/schema#', 'type': 'object', 'properties': {'item_id': {'type': 'string'}, 'timestamp': {'type': 'string'}, 'target_value': {'type': 'number'}, 'ao': {'type': 'number'}, 'jaw': {'type': 'number'}, 'teeth': {'type': 'number'}, 'lips': {'type': 'number'}, 'fdbb': {'type': 'number'}, 'price_peak_bellow': {'type': 'integer'}, 'ao_peak_bellow': {'type': 'integer'}}, 'required': ['ao', 'ao_peak_bellow', 'fdbb', 'item_id', 'jaw', 'lips', 'price_peak_bellow', 'target_value', 'teeth', 'timestamp']}


In [124]:
import json
# Convert the dictionary to a JSON string
schema_json = json.dumps(buildder_schema_data)

# Write the JSON string to the file
with open(fn_basename+ '.schema.json', 'w') as f:
  f.write(schema_json)


## Issue Format in AWS Forecasting import


To convert the JSON schema to a format that AWS Glue expects, you can use Python. AWS Glue expects a list of dictionaries, where each dictionary represents a column in the schema. Each dictionary should have two keys: 'Name' and 'Type'. The 'Name' key corresponds to the column name, and the 'Type' key corresponds to the column type.

Here's a Python script that converts the JSON schema to the AWS Glue format:

```python
import json

# Load the JSON schema
with open('SPX500_D1.targets.mx.schema.json', 'r') as f:
    schema = json.load(f)

# Initialize an empty list to hold the AWS Glue schema
glue_schema = []

# Convert the JSON schema to the AWS Glue format
for name, properties in schema['properties'].items():
    glue_schema.append({
        'Name': name,
        'Type': properties['type']
    })

# Print the AWS Glue schema
print(glue_schema)
```

This script will print a list of dictionaries, where each dictionary represents a column in the AWS Glue schema. The 'Name' key in each dictionary corresponds to the column name, and the 'Type' key corresponds to the column type.

Please note that this script assumes that the JSON schema is in the same directory as the script and that the file is named 'SPX500_D1.targets.mx.schema.json'. If the file is located elsewhere or has a different name, you will need to modify the script accordingly.

Also, please note that the script maps the JSON schema types directly to the AWS Glue types. If the types in your JSON schema do not match the types expected by AWS Glue, you will need to add a mapping between the JSON schema types and the AWS Glue types.

In [125]:

# Load the JSON schema
import json
with open(fn_basename + '.schema.json', 'r') as f:
    schema = json.load(f)

# Initialize an empty list to hold the AWS Glue schema
glue_schema = []

# Convert the JSON schema to the AWS Glue format
for name, properties in schema['properties'].items():
    glue_schema.append({
        'Name': name,
        'Type': properties['type']
    })

# Print the AWS Glue schema
print(glue_schema)


[{'Name': 'item_id', 'Type': 'string'}, {'Name': 'timestamp', 'Type': 'string'}, {'Name': 'target_value', 'Type': 'number'}, {'Name': 'ao', 'Type': 'number'}, {'Name': 'jaw', 'Type': 'number'}, {'Name': 'teeth', 'Type': 'number'}, {'Name': 'lips', 'Type': 'number'}, {'Name': 'fdbb', 'Type': 'number'}, {'Name': 'price_peak_bellow', 'Type': 'integer'}, {'Name': 'ao_peak_bellow', 'Type': 'integer'}]


In [126]:
# Write the AWS Glue schema to a file
with open(fn_basename + '.glue.json', 'w') as f:
    json.dump(glue_schema, f, indent=2)




The error message suggests that the AWS service you're using does not expect the schema to be a list of dictionaries. Instead, it seems to expect a single dictionary where each key is a column name and each value is the corresponding column type.

Here's a Python script that converts the AWS Glue schema to this format:

```python
import json

# Load the AWS Glue schema
with open('SPX500_D1.targets.mx.glue.json', 'r') as f:
    glue_schema = json.load(f)

# Initialize an empty dictionary to hold the converted schema
converted_schema = {}

# Convert the AWS Glue schema to the expected format
for column in glue_schema:
    converted_schema[column['Name']] = column['Type']

# Print the converted schema
print(converted_schema)
```

This script will print a single dictionary where each key is a column name and each value is the corresponding column type.

Please note that this script assumes that the AWS Glue schema is in the same directory as the script and that the file is named 'SPX500_D1.targets.mx.glue.json'. If the file is located elsewhere or has a different name, you will need to modify the script accordingly.

In [127]:


import json

# Load the AWS Glue schema
with open(fn_basename + '.glue.json', 'r') as f:
    glue_schema = json.load(f)

# Initialize an empty dictionary to hold the converted schema
converted_schema = {}

# Convert the AWS Glue schema to the expected format
for column in glue_schema:
    converted_schema[column['Name']] = column['Type']

# Print the converted schema
print(converted_schema)

attributes = []
for name, data_type in converted_schema.items():
    if name == 'timestamp':
        data_type= "timestamp"

    if name == 'target_value':
        data_type = 'float'    
    
    if name == 'item_id':
        data_type = 'string'
    
    if name != 'timestamp' and name != 'target_value' and name != 'item_id':
        data_type = 'string'
    
                 
    attributes.append({
        "AttributeName": name,
        "AttributeType": data_type
    })

formatted_attributes = {"Attributes": attributes}

formatted_attributes






{'item_id': 'string', 'timestamp': 'string', 'target_value': 'number', 'ao': 'number', 'jaw': 'number', 'teeth': 'number', 'lips': 'number', 'fdbb': 'number', 'price_peak_bellow': 'integer', 'ao_peak_bellow': 'integer'}


{'Attributes': [{'AttributeName': 'item_id', 'AttributeType': 'string'},
  {'AttributeName': 'timestamp', 'AttributeType': 'timestamp'},
  {'AttributeName': 'target_value', 'AttributeType': 'float'},
  {'AttributeName': 'ao', 'AttributeType': 'string'},
  {'AttributeName': 'jaw', 'AttributeType': 'string'},
  {'AttributeName': 'teeth', 'AttributeType': 'string'},
  {'AttributeName': 'lips', 'AttributeType': 'string'},
  {'AttributeName': 'fdbb', 'AttributeType': 'string'},
  {'AttributeName': 'price_peak_bellow', 'AttributeType': 'string'},
  {'AttributeName': 'ao_peak_bellow', 'AttributeType': 'string'}]}

In [128]:

# Write the converted schema to a file

with open(fn_basename + '.glue.converted.f.json', 'w') as f:
    json.dump(formatted_attributes, f, indent=2)
    

In [129]:
# Write the converted schema to a file

with open(fn_basename + '.glue.converted.json', 'w') as f:
    json.dump(converted_schema, f, indent=2)
    

## Keep only those attributes

```json

{
  "Date": "string",
  "Close": "number",
  "ao": "number",
  "ac": "number",
  "jaw": "number",
  "teeth": "number",
  "lips": "number",
  "fh": "integer",
  "fl": "integer",
  "fdbb": "number",
  "price_peak_bellow": "integer",
  "ao_peak_bellow": "integer",
  "target": "number"
}

```